# EP Viscoelastic model
Viscoelastic version of the standard finite element model. Uses customised version of libraries

In [ ]:
#%reset -f
#import sys
#original_path = sys.path.copy()
#sys.path.append('..')
from lib_fem import *
#from lib.lib_plot import *
#sys.path = original_path

'''Pre-init'''

#empty cache
os.system('rm -rf /root/.cache/fenics/')

'''Scalings'''
# Length scale: mm
# Time scale: hours
# The numerical code is entirely dimensional.

###############################

'''Parameters'''

# Length scales (all in mm)
Er = 1.81  # embryo radius
r = .92 # margin radius
d = .12 # margin thickness
offset = .106 #initial downward displacement of EP in mm

mu = 1 # shear viscosity
baseline_tension = 2.5*mu #time-independent baseline tension on margin
t_dep = 'plat' # 'const', 'plat', 'lin' 

hypothesis = "yield" # "apical_constriction", "basal_tension", "yield"
K0 = 10 # Elasticity scale (viscosity/time)
g_b = 5 # Basal tension at edge
r0 = 0.55 #equilibrium basal length
L_a = 5 #apical A-M contractility
alpha_l = 4 #cell-cell adhesion
marg_ampl = 2 #amplification of L_a at margin

ablation_time = 6
HR = 20 # factor higher time resolution after ablation

# Regulation settings
t0 = 0                     # Start time
t1 = ablation_time+20      # Final time
dt = 0.01                  # Time step size
nrep = int(1 + ablation_time/dt + 2*mu/K0/dt*HR) #int((t1-t0)/dt)+1
subd=1
remeshinterval = 25 #remesh after this many FEM timesteps
plotinterval = 50

N = 64 #number of margin segments, choose even
# output data directory
dataDir='output/'

# switches
regulate = 0 # if 0 tension has fixed gaussian profile WARNING regulate=0 has not yet been tested
remesh = 1 #remesh WARNING REMESHING RESETS THE LOCUS OF THE HAIR (IF PRESENT)
hair = 0 #hair friction on/off.
echo = 0 #print margin variables at each timestep
output = 1 #flag for saving output. 1 = on, 0 = off

ten_profile = "syn" # "syn", "unif"

# numerical parameters
meshres = .1 #mesh resolution in mm
pen = 2000 #penalty for weak enforcement of no slip boundary condition if divergence is non-zero, choose large
THorder = 1 #order of the FE method (choose at least 1. Velocity elements have degree THorder + 1)


############################


'''legacy parameters, have no effect but need to be passed'''
l_diff = 0.1583 # diffusion length
# Time scales
tau = 0.5 # myosin regulation time scale [hrs]
emax = 1/3 # maximal contraction rate [1/hrs]
ve_time = 0.5 # margin viscoelastic time scale [hrs]
# Myosin scale
c0 = 1 #set to 1 (unknown units).
# Tension scale
Ts = 1 #set to 1 (unknown number of Newtons per unit myosin)
# Dimensionless biological parameters (controlling myosin regulation in response to stretching)
alpha = -1.0 # regulation offset
beta = 16.0 # sensitivity of myosin to stretching
zeta = 0.8/3 #1/3 # amplitude of myosin variation
lam  = 5.0 # #sensitivity of walkers to tension
# Hair parameters
fr = 500 #hair friction coefficient
xmin = 0 #-20 #minx for hair
# Tissue viscosity scale
#mu = 2/30 #0.1 # 2D tissue viscosity ( in units of [c0*Ts] * hours / mm )
# Initialisation parameters
Camp = c0*zeta #initial myosin amplitude (perturbation from equilibrium)
initialT = c0*Ts #c0*Ts #initial tension (uniform along margin)


'''Wrappers'''
# wrap parameters for easy query. All of these are immutable
#
general_parameters = [N,t0]
regulation_parameters = [alpha, beta, zeta, lam, emax, tau, l_diff, ve_time, c0, Ts]
initialisation_parameters = [c0,Camp,initialT, zeta, alpha]
tissue_parameters = [mu,fr*hair]
geometry_parameters = [Er,r,d,offset]
el_parameters = [r0, L_a, g_b, alpha_l, K0, marg_ampl]
#
regulation_settings = [dt,subd,regulate,remeshinterval*remesh]    
fem_settings = [meshres,pen,THorder]
#
flags = [regulate,remesh,hair,echo,output]
    
"""Main program"""

'''Initialisation'''

t = t0 #current times is the initial time
currentradius = Er #current radius is the initial radius
imagestack = [] #empty imagestack for plot output
fig = None
ax = None

#prepare output
if output:
    dataout = prepare_output(dataDir)
    save_settings(dataDir,general_parameters,regulation_parameters,initialisation_parameters,tissue_parameters,geometry_parameters,regulation_settings,fem_settings,flags)
    
#initialise margin
margin_geom, ps, l, lt, cumt, th, ta, c, T, interpolators = initialise_mso(N,t,initialisation_parameters,geometry_parameters,regulate,mu,ten_profile,baseline_tension)

#mesh embryo
mesh, meshtags, markers = mso_meshing(N,meshres,currentradius,ps) #EP,EE,hair have indices 1,2,3 respectively

#Identify mesh regions with area growth (keeping for calculation of edge velocity)
Ms, coefs = epiboly_init(mesh,geometry_parameters,THorder,t)
acc_ing = accum_ingression(t,mesh)

#Initialise solver (important to do this outside of the loop)
solver = create_solver()

#Create function spaces for velocity and density
V, R = create_function_spaces(mesh)

#Define initial conditions
U, Rho, v_sol = define_initial_conditions(V,R)
Dv = calc_div(mesh,U,R)

In [ ]:
'''Loop'''
for j in range(nrep):
    print(f'{t:.4f}')

    if np.isclose(t,ablation_time):
        #reduce time step to 1/10th of the original time step after edge is cut
        dt = dt / HR
        plotinterval = HR
    
    '''Solve FEM'''

    #Calculate active stress object
    stressobj = get_stressobj(margin_geom,np.min([t,ablation_time]),N,mu,d,ten_profile,baseline_tension,interpolators,l,cumt,t_dep)
    stressobj1D = get_stressobj1D(margin_geom,np.min([t,ablation_time]),N,mu,d,ten_profile,baseline_tension,interpolators,l,cumt,t_dep)

    #obtain velocity and accumulated area changes
    v, vn = VE_femsolve(t,U,V,N,mesh,markers,Ms,coefs,stressobj,stressobj1D,acc_ing,tissue_parameters,fem_settings,el_parameters,hypothesis,ablation_time,solver)
    if not remesh:
        v_sol.x.array[:] = v.x.array[:]
    
    '''Process solution'''    
    #Evaluate v at margin points (using redefined margin)
    v_values = vel_on_margin(v,ps,mesh,N)
    
    if j==0:
        vtxs = init_writers(dataDir,U,Rho,v_sol,Dv,mesh)
    
    #Calculate extension rate
    margvel, gd = calc_exrate(v_values,ta,l,N)

    '''Save and display output'''

    #save output
    if output:
        save_fem(mesh,str(j),dataDir,U,v,Rho,Dv) #saves velocities, densities, and displacement to xdmf files
        dataout = save_marg(dataout,t,T,c,gd,l,ps,margvel)
        
        #write to vtx
        for vtx in vtxs:
            vtx.write(t)

    #plot flow and divergence
    if (j % plotinterval == 0):
        #plot_flow_pyvista(mesh,v,V,scale=2,newfig = not j) #plot velocity
        #plot_div_pyvista(v,None,mesh,1,option=0,THorder=1)#,clim=[-0.5, 0.5]) #instantaneous divergence of the flow
        #plot_p_pyvista(Dv,mesh,1,THorder=1)#,clim=[0.8, 1.2]) #accumulated area changes
        pass

    '''Time step margin'''

    #time-step margin coordinates, myosin and tension
    ps, c, T = regulate_margin(N,ps,v_values,gd,c,lt,T,None,None,None,regulation_parameters,regulation_settings,dt,mu,ten_profile,baseline_tension,t)

    #update quantities derived from new margin position
    margin_geom, l, lt, cumt, th, ta = updatelengths(ps,N)
    interpolators = interp_init(cumt,c,T,th)
    
    '''Time step mesh (remesh if desired)'''
    
    for sdiv in range(subd):
        ddt = dt/subd
        Rho, U = evolve_densities_and_displacement(mesh,U,Rho,R,v,ddt)
        Dv = calc_div(mesh,U,R,Dv)
        #move mesh and update radius
        mesh = move_mesh(mesh,v,ddt)
        #Update radius
        currentradius+=ddt*vn 
    
    #remesh if desired
    if remesh and ((j+1) % remeshinterval == 0) and (t < ablation_time):
        #Redefine margin
        margin_geom, ps, l, lt, cumt, th, ta, c, T, interpolators = redef_margin(N,margin_geom,l,cumt,interpolators)
        #Redefine mesh
        currentradius = det_newradius_retract(mesh) #trim currentradius to fit new mesh into old mesh
        newmesh, meshtags, markers = mso_meshing(N,meshres,currentradius,ps)
        #Interpolate Epiboly and state variables
        Ms = epiboly_remesh(newmesh,mesh,Ms,THorder)
        U = displacement_remesh(newmesh,mesh,U,THorder)
        Rho = density_remesh(newmesh,mesh,Rho,THorder)
        #Define new function spaces
        V, R = create_function_spaces(newmesh)
        Dv = calc_div(newmesh,U,R) #re-calculate divergence just in case
        #drop old mesh
        mesh = newmesh
   
    #update time step
    t += dt

    #Update contraction coefficients
    coefs = update_coefs(mesh,t)
    acc_ing = accum_ingression(np.min([t,ablation_time]),mesh)

#Close output files
for vtx in vtxs:
    vtx.close()

save_output(dataDir,dataout,'Margin_data')